# setup api and dataset

In [1]:
!pip install goodfire==0.2.37

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
try:
  from google.colab import userdata

  # Add your Goodfire API Key to your Colab secrets
  GOODFIRE_API_KEY = userdata.get('GOODFIRE_API_KEY')
except:
  keyfile = open("../goodfire_apikey.txt", "r")
  GOODFIRE_API_KEY = keyfile.readline()

import goodfire
client = goodfire.Client(
    GOODFIRE_API_KEY
  )

# Instantiate a model variant
variant_small = goodfire.Variant("meta-llama/Meta-Llama-3-8B-Instruct")
variant_large = goodfire.Variant("meta-llama/Meta-Llama-3.1-70B-Instruct")

In [3]:
# first try to install datasets
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# try tqdm (it doesnt seem to work for some reason)
!pip install tqdm
from tqdm.notebook import tqdm
import time

for i in tqdm(range(100)):
    time.sleep(0.01)

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
# next try to download the triva dataset
from datasets import load_dataset

ds = load_dataset("mandarjoshi/trivia_qa", "rc")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/138384 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17210 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

# Run the Baseline

In [7]:
# try running through all of the training set to see what performance we get at base
variant_small.reset()
prompt = "You are an AI assitant answering trivia questions. Give a single concise answer."
num_correct = 0
timeout = 0.5
num_questions = 0
for index, sample in enumerate(tqdm(ds["train"])):
    sample_question =  sample["question"]
    sample_answers = sample['answer']['normalized_aliases']

    if len(sample_answers[0]) > 50:
            continue

    # now get the model response
    response = client.chat.completions.create(
        [
            {"role": "system", "content": prompt},
            {"role": "user", "content": sample_question}
        ],
        model=variant_small,
        stream=False,
        max_completion_tokens=50,
    )

    given_answer = response.choices[0].message["content"].lower()

    #given_answer = ""
    #for token in response:
    #    given_answer += token.choices[0].delta.content
    #given_answer = given_answer.lower()

    for answer in sample_answers:
        if answer in given_answer:
            num_correct += 1
            break
    num_questions += 1

    if num_questions > 300:
        break

    # make sure not to spam the api
    time.sleep(timeout)


  0%|          | 0/138384 [00:00<?, ?it/s]

In [8]:
print(num_correct)
print(num_questions)
print(num_correct/num_questions)
#150
#301
#0.4983388704318937

181
301
0.6013289036544851


In [9]:
# try running through all of the training set to see what performance we get at base
variant_large.reset()
prompt = "You are an AI assitant answering trivia questions. Give a single concise answer."
num_correct = 0
timeout = 0.5
num_questions = 0
for index, sample in enumerate(tqdm(ds["train"])):
    sample_question =  sample["question"]
    sample_answers = sample['answer']['normalized_aliases']

    if len(sample_answers[0]) > 50:
            continue

    # now get the model response
    response = client.chat.completions.create(
        [
            {"role": "system", "content": prompt},
            {"role": "user", "content": sample_question}
        ],
        model=variant_large,
        stream=False,
        max_completion_tokens=50,
    )

    given_answer = response.choices[0].message["content"].lower()

    #given_answer = ""
    #for token in response:
    #    given_answer += token.choices[0].delta.content
    #given_answer = given_answer.lower()

    for answer in sample_answers:
        if answer in given_answer:
            num_correct += 1
            break
    num_questions += 1

    if num_questions > 300:
        break

    # make sure not to spam the api
    time.sleep(timeout)


  0%|          | 0/138384 [00:00<?, ?it/s]

In [10]:
print(num_correct)
print(num_questions)
print(num_correct/num_questions)
#183
#301
#0.6079734219269103

231
301
0.7674418604651163
